# Lattice Boltzmann method in 2D - Computational Fluid Dynamics Exercise
The [lattice Boltzmann methods](https://en.wikipedia.org/wiki/Lattice_Boltzmann_methods) (LBM), is a class of computational fluid dynamics (CFD) methods for fluid simulation. Instead of solving the Navier–Stokes equations directly, a fluid density on a lattice is simulated with streaming and collision (relaxation) processes.
This is made following this [YouTube tutorial](https://www.youtube.com/watch?v=JFWqCQHg-Hs), which is in turn based on this [Medium post](https://medium.com/swlh/create-your-own-lattice-boltzmann-simulation-with-python-8759e8b53b1c).

In [2]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

Demonstrating the functionalities of some of the parts below and a few basic numpy uses.

`X, Y = np.meshgrid(<2 linear arrays>)`

*Iterating over `X[i][j], Y[i][j]` gives x, y coordinates of row `i` (y all equal) and col or 'row entry' `j` (x all equal)*

It makes sense to differentiate between the implicit coordinates given by the shape and indices of the arrays, where the outer array's index correspond to the y coordinates, and the inner arrays' indices correspond to the x coordinates.

The values of the inner arrays are the row entries of a table specifying what value is associated with that implicit coordinate.
In the case of the meshgrid, we have two different "datasets", so to speak. One for the x coordinate values at each (implicit) x-y coordinate point and one for the y coordinate values. Because we have two datasets, `np.meshgrid` returns them as a list, not an array (I think). When viewed like this, it's less confusing that there are two ways of talking about the coordinate grid within one object, namely the coordinate array (of which there are two, which also doesn't help). The implicit, index-based coordinates are the "container", all else within are arbitrary data associated with that point.

In [3]:
# meshgrid for 2D functions
axis = np.linspace(0,1,100+1)
X, Y = np.meshgrid(axis, axis)
print('X', X)
print('Y', Y)

# a random function with two variables
f = np.sin(X*10) * np.cos(Y*10)
# specific point for illustration
x, y = 10, 30
print(f[x,y])

# Plotting a heatmap. Alternatively, go.Contour works the same.
go.Figure(data=[go.Heatmap(z=f),go.Scatter(x=[x],y=[y],name='x, y')]).update_layout(width=500, height=500, xaxis_scaleanchor='y', xaxis_scaleratio=1) 

X [[0.   0.01 0.02 ... 0.98 0.99 1.  ]
 [0.   0.01 0.02 ... 0.98 0.99 1.  ]
 [0.   0.01 0.02 ... 0.98 0.99 1.  ]
 ...
 [0.   0.01 0.02 ... 0.98 0.99 1.  ]
 [0.   0.01 0.02 ... 0.98 0.99 1.  ]
 [0.   0.01 0.02 ... 0.98 0.99 1.  ]]
Y [[0.   0.   0.   ... 0.   0.   0.  ]
 [0.01 0.01 0.01 ... 0.01 0.01 0.01]
 [0.02 0.02 0.02 ... 0.02 0.02 0.02]
 ...
 [0.98 0.98 0.98 ... 0.98 0.98 0.98]
 [0.99 0.99 0.99 ... 0.99 0.99 0.99]
 [1.   1.   1.   ... 1.   1.   1.  ]]
0.07624746575887673


The basic setup of LBM can be described as a set of microscopic particles that can move only on a grid and in discrete steps. By simulating *streaming* and *colliding* of the particles, and tuning parameters for the statistics the right way, we can get them to behave like a fluid macroscopically, but with much lower computational requirements.

The version of the model used here is called *D2Q9*, for two dimensions and 9 velocities. At one node of the lattice there are 9 possibilities for the value the speed of the fluid particle can have, in particular in which direction the velocity vector points. Ther is one for staying put, i.e. zero velocity, 4 for neighbouring lattice nodes or grid cells, and 4 for diagonal neighbours, touching the cell at the corner (or edge in 3D). It seems lattice, grid, cells, nodes, etc. are being thrown around pretty willy-nilly, and the neighbouring nodes are identified with the discrete velocities, which are in turn identified with the particles somehow, at least in the two sources cited above. I'll try to keep it straight here.

In [14]:
# meshgrid for combinations of directions
X, Y = [coord_arr.flatten() for coord_arr in np.meshgrid([-1,0,1], [-1,0,1])]
navier_stokes_values = ['1/36', '1/9', '1/36',
                        '1/9', '4/9', '1/9',
                        '1/36', '1/9', '1/36']
(px.scatter(x=X, y=Y, color=range(9))
 .update_layout(xaxis_scaleanchor='y', xaxis_scaleratio=1, width=500, height=400, 
                annotations=[dict(x=x,y=y, text=t) for x,y,t in zip(X,Y,navier_stokes_values)],
                coloraxis_colorbar=dict(title='order in numpy array')
                )
)

### Distribution function
One of the central parts of the simulation is the so called *distribution function* `F` ($f$ for now, as we talk about the continuous case first), which tells us how many particles we have at some location x,y (i.e. their density) and some time t. Specifically, it tells us about the density in *phase space*, which includes the velocity as an additional dimension. 

`F` / $f$ could also be called an equation of state (I believe), but it is more like a lookup table. At least for me, the term function is confusing here, as it has `Nx + Ny + 1 + N_lattice` input variables and no "machinery" to speak of. The machinery that evolves it forward in time is what we are building here.

(Okay, it is called distribution function, because originally it is the probability distribution for a single particle in phase space. The continuous Boltzmann equation describes its evolution.)


### Mechanism
Streaming and colliding are the two mechanisms by which we step forward in time, using something called the *BKG approximation* (Bhatnagar Gross and Krook), an equation: 

$$ (\partial_t + \bold{v} \cdot \nabla) f = -\frac{f-f^{eq}}{\tau} $$

Left represents streaming, right colliding. More on what that means below. First, we have to make this easy to compute by making it work with the lattice idea. Concretely, we want to *"discretize"* the equation. For each $i$ of the 9 lattice directions (or velocities, as zero is included) this could look like this:

$$ F_i(\bold{x}_i + \bold{v}_i \Delta t, t + \Delta t) - F_i(\bold{x}_i, t) = -\frac{F_i(\bold{x}_i,t)-F_i^{eq}(\bold{x}_i,t)}{\tau} $$

This more or less says: "At each lattice point go through all of the 9 directions. On the streaming side of the equation, increase the density based on the velocity (times the time step length), then subtract the previous density state, to get what changed. That should be equal to the (negative of the) collision side, where we do something similar. But instead of updating the density via a velocity, we update it by the difference of the current density state to the equilibrium state. Then we scale that update based on the timescale."

(I guess it's best to think of the velocity as particles per unit time, not distance per unit time, so it's more like a mass flow density. Also, I guess this equation is based on conservation of mass / particles or something.)


------------------------------

Okay, maybe the way the equation is written above is not that helpful for understanding? If we do it like this:
$$ F_i(\bold{x}_i + \bold{v}_i \Delta t, t + \Delta t) = F_i(\bold{x}_i, t) +\frac{F_i^{eq}(\bold{x}_i,t) - F_i(\bold{x}_i,t)}{\tau} $$
it seems to say that the change of the state is exactly what changes from the collision part of the model? Okay, but that was clear from the beginning, I mean it's an equation, so these are equal.

Hm, I'm confused at the moment, but I have the intution that this still is a sensible way to look at things. Below, I'm talking/wondering about where and what we stream around, and then what will intervene to make the streaming particles not go on in one direction forever. Anyway, let's continue.

------------------------------

We can then use $F$ (`F` in the code) to ask questions about the whole simulation area over time, or of course at each lattice node / grid cell. We do this by calculating something with $F_i$ and then summing over all lattice directions $i$. Just taking $F$ gives us the density, multiplying by $\bold{v}_i$ gives us the momentum. Doing these calculations means getting the *moments of the distribution function*.

Now, let's look in some more detail at what the equation actually represents, or how we step it forward through time.



### Streaming

The tricky thing to understand here is how the small 3x3 lattice relates to the big lattice or grid. At each time step, we shift all of the values in $F$ around on the big lattice. Specifically, for each value in the small lattice, we shift it in the direction represented by that point on the small lattice. Where do we shift it? Into the respective cell in that direction I guess (we will probably do something to it, so that it doesn't just continue in that direction, collisions probably). What do we shift? An amount of particles, as described by the density, or a bit of fluid density. The fraction of the particles in that cell that happen to move in that specific direction.

Wikipedia says: *Then, for example vector ${\displaystyle {\vec {e}}_{4}=(0,-1)}$, i.e., it points due south and so has no $\displaystyle x$ component but a $\displaystyle y$ component of $\displaystyle -1$. So one of the nine components of the total density at the central lattice point, $\displaystyle f_{4}({\vec {x}},t)$, is that part of the fluid at point $\displaystyle {\vec {x}}$ moving due south, at a speed in lattice units of one.*

For simplicity, we just use one without a unit for the time step length and the lattice distance (there is probably a good reason for why we can ignore that stepping diagonally is longer than the other directions).

All of this is basically just following from the definition of $F$ itself: Wikipedia: *As $\displaystyle f_{i}({\vec {x}},t)$ is, by definition, the fluid density at point $\displaystyle {\vec {x}}$ at time $\displaystyle t$, that is moving at a velocity of $\displaystyle {\vec {e}}_{i}$ per time step, then at the next time step $\displaystyle t+\delta _{t}$ it will have flowed to point $\displaystyle +{\vec {e}}_{i}$.*

(By common convention, the lattice speed $c$ is set to 1, but it is defined by the soundspeed (the square of the soundspeed is 1/3).)

### Colliding

The sources above give us this for the colliding part, a simplification of a fluid using the assumption of constant temperature and constant sound speed.  So this is the equilibrium state of $F_i$:

$$ F_i^{eq} = w_i \rho \left( 1 + 3 (\bold{v}_i \cdot \bold{u}) + \frac{9}{2}(\bold{v}_i \cdot \bold{u})^2 + \frac{3}{2}(\bold{u}\cdot\bold{u})^2\right) $$

where $u$ is the *macroscopic* or net velocity at point x,y. (This is actually quite important, although it hasn't appeared so far.) 

Roughly, the equilibrium is based on the idea that for a bunch of particles, we may not know the magnitudes of the individual speeds, just the mean speed, as that is related to temperature. But using the central limit theorem, we indeed can know the distribution (actually the famous Maxwell-Boltzmann distribution apparently!), which is basically normal. Then we do a Taylor approximation at the relevant point in velocity space (2nd order), instead of using the actual distribution.

So we now know the equilibrium distribution of speeds, and we can compare that to the current distribution (for each point on the lattice) and because we have the time scale $\tau$ defined as the time it takes for the fluid to relax to the equilibrium speed distribution, we can take the velocity change as the difference between the current state and the equilibrium state per $\tau$.

In the equation above, we're not seeing c = sqrt(3RT), presumably because it is set to 1. I don't know why this is the lattice speed, but sqrt(J) is at least sqrt(mass) length/time, and the c's are all to the power of 2 or 4 and there is $\rho$ outside the parentheses, to cancel the mass. I guess something else is set to 1 with the density to make it work. 🤷 Anyway, the weights we've seen above also fall out of this derivation from the MB-distribution.

Okay, quick attempt to summarize it all: We have a lattice along which a bunch of particles can move. The speeds are discrete, like the positions, and can take on one of 9 values. So at each lattice node, there are densities of particles with different speeds, specifically speeds in different directions. When we crank the system forward, the particles move according to these speeds (streaming). If we add relaxation to some equilibrium speed distribution, the particles start moving differently. The relaxation is based on how the speeds should be distributed if they collide randomly and have some mean speed based on temperature.

In [120]:

def run_lattice():
    # grid size in x and y direction
    Nx = 400
    Ny = 100

    # average density
    #rho0 = 100

    # kinematic viscosity / time scale
    tau = 0.53

    # time steps
    Nt = 60000
    #Nt = 15

    # when to sample F
    plot_every = 50

    # 3x3 lattice for speeds at each node, in the x-y plane
    # 8 potential directions to go, +1 for staying stationary
    N_lattice = 9
    # How to step along x, y coordinates for each speed (direction)
    # all combinations of potential x and y values of the lattice velocity c
    xs_lat_vel, ys_lat_vel = [coord_arr.flatten() for coord_arr in np.meshgrid([-1,0,1], [-1,0,1])]

    # see above for where these come from
    weights = [1/36, 1/9, 1/36,
               1/9,  4/9, 1/9,
               1/36, 1/9, 1/36]
    

    # Initial conditions
    # array to define simulation area, flow density of 1 for each speed
    # Ny rows for Ny cells in height, 
    # Nx entries in each row for Nx cells in length, 
    # each entry has a 3x3 lattice, i.e. 9 values per cell
    F = np.ones([Ny, Nx, N_lattice])

    # add random fluctuation for initialization
    np.random.seed(0)
    F += 0.001 * np.random.randn(Ny, Nx, N_lattice)

    # initial flow is to the right, so in each row of cells, in each cell, 
    # set particle flow density to the right (5th index, 6th entry), to a higher value
    F[:,:,5] = 2.3


    # adding an obstacle (cylinder) after a quarter of the length of the simulation area
    cx, cy = Nx//4, Ny//2
    # all (x,y) coordinates of the cells, as two arrays
    X, Y = np.meshgrid(np.linspace(0,Nx, Nx), np.linspace(0, Ny, Ny))
    # a function to calculate the distance from a point for a set of points
    distance_f = lambda x, y, x0, y0: np.sqrt((x0 - x)**2 + (y0-y)**2)
    # calculate the distances to the center of the cylinder for each grid point
    D = distance_f(X,Y,cx,cy)
    # array for each cell, set to True where D < 15, else False, to define cylinder
    cylinder = np.where(D < 10, True, False)
    cylinder2 = np.where(D < 12, True, False)

    density = np.sum(F, axis=2)
    uy = np.sum(F * ys_lat_vel, 2) / density
    ux = np.sum(F * xs_lat_vel, 2) / density
    uy[cylinder2] = 0
    ux[cylinder2] = 0

    # main loop
    #frames = []
    zs = []
    for step in range(Nt):
        print(step, end=' ')

        # stream (shift) all fluid densities to their respective neighbours
        # at each cell, apply the respective velocity(direction) to each lattice node, which is equivalent to shifting it one cell over, as the lattice speed c is 1
        for node, lv_y, lv_x in zip(range(N_lattice), ys_lat_vel, xs_lat_vel):
            F[:, :, node] = np.roll(F[:, :, node], lv_y, axis = 0)
            F[:, :, node] = np.roll(F[:, :, node], lv_x, axis = 1)
        
        # Invert all velocities where a node touches the cylinder, so they move backwards
        boundary_F = F[cylinder, :]
        boundary_F = boundary_F[:, ::-1]

        # fluid variables
        # density rho via sum over last access of F, axis 2 (i.e. over all directions in the 3x3 lattice)
        density = np.sum(F, axis=2)
        # bulk / net speed
        # F.shape is Ny, Nx, 9 and _lat_vel are both 9, so numpy knows what to do (broadcasting)
        # the multiplication selects for the y / x values respectively, because they are 0 elsewhere.
        # The fully count the diagonally moving particles, because their speed in the relevant direction is still 1
        # because of negative and positive values in _lat_vel opposite speeds cancel out. 
        # Dividing by density gives the average (i.e. macroscopic) speed per unit of fluid density / particle
        uy = np.sum(F * ys_lat_vel, 2) / density
        ux = np.sum(F * xs_lat_vel, 2) / density

        # no movement in the cylinder
        F[cylinder, :] = boundary_F
        uy[cylinder] = 0
        ux[cylinder] = 0

        # collisions
        
        u_val_0 = (ux**2 + uy**2)
        u_val = 3 * u_val_0/2
        F_equilibrium = np.zeros(F.shape)
        for i, lv_y, lv_x, w in zip(range(N_lattice), ys_lat_vel, xs_lat_vel, weights):
            val = 3 * (lv_x*ux + lv_y*uy) # calculate only once
            F_equilibrium[:, :, i] = density * w * (
                1 + val + val**2 / 2 - u_val
            )
            # F_equilibrium[:, :, i] = density * w * (
            #     1 + 3 * (lv_x*ux + lv_y*uy) + 9 * (lv_x*ux + lv_y*uy)**2 / 2 - 3 * (ux*82 + uy**2)/2
            # )
        
        #F = F + -(1/tau) * (F - F_equilibrium)
        F = F - (F - F_equilibrium)/tau

        if step % plot_every == 0:
            z = np.sqrt(u_val_0)
            zs.append(z)
            # frame = go.Frame(
            #     data=[go.Heatmap(z=np.sqrt(ux**2+uy**2), zmin=0, zmax=.25 )])
            # frames.append(frame)


    #return frames
    return zs
#frames = run_lattice()
zs = run_lattice()
 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [121]:
import sys
sys.path.insert(1,'../')
import utils.utils as utils

In [122]:
frames = []
for z in zs:
    frame = go.Frame(
        data=[go.Heatmap(z=z, zmin=0, zmax=.25 )]
    )
    frames.append(frame)

def plot(frames):
    fig = go.Figure(
        data=frames[0].data,
        frames=frames,
        layout=go.Layout(
            xaxis=dict(range=[0, 400], autorange=False),
            yaxis=dict(range=[0, 100], autorange=False),
            xaxis_scaleanchor='y', xaxis_scaleratio=1,
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(
                    label="Play",
                    method="animate",
                    args=[
                         None,
                        {'frame':{
                            'duration':16.6*3,
                            'transition':{
                                'duration':0
                            }
                        }}
                    ]
                )]
            )]
        ),
    )
    return fig
    #fig = go.Figure(data = frames[2]['data'])
    #print(frames)

#if __name__ == '__main__':
#    main()

fig = plot(frames[:])
utils.saveFig(fig, 'LBM')
fig.show()

KeyboardInterrupt: 

In [62]:
np.random.seed(0)
N = 120
data = np.random.random(size=[10,10])*10
frames = [
    go.Frame(
        data=go.Heatmap(
            z=np.where(data-0.05*n >=0, data-0.05*n, 0),
            zmin=0, zmax=10
        ),
        layout=go.Layout(
            xaxis=dict(
                range=[0, 10],
                autorange=False
            ),
            yaxis=dict(
                range=[0, 10],
                autorange=False
            ),
        )
    ) for n in range(N)
]
go.Figure(
    data=go.Heatmap(z=data),
    layout=go.Layout(
        xaxis=dict(range=[0, 10], autorange=False),
        yaxis=dict(range=[0, 10], autorange=False),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(
                label="Play",
                method="animate",
                args=[
                     None,
                    {'frame':{
                        'duration':16.6,
                        'transition':{
                            'duration':0
                        }
                    }}
                ]
            )]
        )]
    ),
    frames=frames
)
